In [254]:
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix
import sys 
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, ClusterCentroids
from imblearn.ensemble import EasyEnsembleClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix
from sklearn.datasets import make_classification

In [255]:
holdout_GP = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/holdout_test/GP/'
og_GP = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/'
holdout_RF_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/holdout_test/RF/'
og_RF = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'
original_RF = pd.read_csv(og_RF+'RF_test_results_all_NEK.csv')
original_RF['set'] = 'original_RF'

holdout_RF = pd.read_csv(holdout_RF_path+'RF_holdout_results.csv')
holdout_RF['set'] ='literature_RF'
columns = ['model','NEK', 'set','strategy', 'feat_type', 'RF_type','model', 'cm', 'recall', 'precision', 'accuracy','specificity',
           'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1']

holdout_RF = holdout_RF[columns]
original_RF = original_RF[columns]

# original_RF = original_RF.reset_index(drop=True)
# holdout_RF = holdout_RF.reset_index(drop=True)
all_rf = pd.concat([original_RF, holdout_RF], axis=1)

all_rf.shape

(224, 32)

48

In [256]:
holdout = pd.read_csv(holdout_GP+'GP_holdout_test_results.csv')
holdout.to_csv(holdout_GP+'GP_holdout_test_results.csv',index=False)
original = pd.read_csv(og_GP+'GP_test_results_all_NEK.csv')
matern_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_matern_kernel/'
gp_matern = pd.read_csv(matern_path+'GP_maternkern_test_results.csv')


In [257]:
gp_matern.shape

(48, 18)

In [258]:
holdout['strategy'] = holdout['strategy'].str.replace('scaled', 'raw')
holdout['model'] = holdout['model'].str.replace('scaled', 'raw')
original['strategy'] = original['strategy'].str.replace('scaled', 'raw')
original['model'] = original['model'].str.replace('scaled', 'raw')
original['set'] = 'original_GP'
holdout['set'] ='literature_GP'
gp_matern['set'] = 'original_GP_matern'
gp_matern['modeling_type'] = 'GP_matern'

columns = ['NEK', 'strategy', 'feat_type', 'model', 'cm', 'recall', 'precision', 'accuracy','specificity', # remove tn fn fp tp
           'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1', 'set']
holdout = holdout[columns]
original = original[columns]


all_gp = pd.concat([original, holdout, gp_matern], axis=0).reset_index(drop=True)
all_gp = all_gp.sort_values(['NEK','strategy'])
nek3 = all_gp[(all_gp['NEK'] == 'NEK3_binding') & (all_gp['modeling_type'] == 'GP_matern')] 

# for nek in all_gp['NEK'].unique(): 
#     print(nek)
#     nek_df = all_gp[all_gp['NEK'] == nek]
#     nek_df = nek_df.sort_values(['strategy', 'model'])
#     display(nek_df)


In [259]:
def average_metric(df, metric): 
    averages = []
    neks = df['NEK'].unique()
    for nek in neks:
        nek_df=df[df['NEK'] == nek] 
        average_score = nek_df[metric].mean() 
        print(f'{nek} average {metric} score: {average_score:.2f}') 
        averages.append(average_score) 
    print()
    return averages

In [222]:
# avg_dfs = []
# for model in all_gp['set'].unique(): 
#     this_df = all_gp[all_gp['set'] == model] 
#     print(f"{this_df['set'].loc[0]}") 
#     f1s = average_metric(this_df, 'f1')
#     MCCs = average_metric(this_df, 'MCC')
#     BAs = average_metric(this_df, 'Balanced Accuracy')
#     ROC_AUCs = average_metric(this_df, 'ROC-AUC')
#     precs = average_metric(this_df, 'precision')
#     accs = average_metric(this_df, 'accuracy')
#     specs = average_metric(this_df, 'specificity')
#     recalls = average_metric(this_df, 'recall')
#     avg_df = pd.DataFrame({'set': model,'recall': recalls, 'accuracy': accs, 'precision': precs, 'specificity': specs, 'f1': f1s,
#                           'MCC': MCCs, 'Balanced Accuracy': BAs, 'ROC-AUC': ROC_AUCs})

#     avg_dfs.append(avg_df)
    

In [223]:
# original_avg = avg_dfs[0]
# literature_avg  = avg_dfs[1]
# # literature_avg.set_index(holdout['NEK'].unique(),inplace=True)
# # original_avg.set_index(original['NEK'].unique(),inplace=True)
# literature_avg['NEK'] = holdout['NEK'].unique()
# literature_avg.insert(0, 'NEK', literature_avg.pop('NEK'))
# original_avg['NEK'] = original['NEK'].unique()
# original_avg.insert(0, 'NEK', original_avg.pop('NEK'))
# avg_df_all = pd.concat([original_avg, literature_avg])
# avg_df_all = avg_df_all.sort_values('NEK')
# avg_df_all.to_excel('comparison_GP_literature_set.xlsx',index=False)

# RF models (no GS) 

In [260]:
def average_metric_by_type(df, group_by): 
    averages = []
    neks = df['NEK'].unique()
    neks_all = []
# capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/proba_dfs
    for nek in neks:
        nek_df=df[df['NEK'] == nek] 
        this_set = nek_df['set'].iloc[0]
        numeric_df = nek_df.select_dtypes(include=['number'])
        grouped = numeric_df.groupby(nek_df[group_by]).mean().reset_index()
        grouped['NEK'] = nek
        grouped['set'] = this_set
        averages.append(grouped)
    # display(pd.concat(averages,ignore_index=True))
    return pd.concat(averages,ignore_index=True) 


In [261]:
holdout_RF_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/holdout_test/RF/'
og_RF = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'
holdout_RF = pd.read_csv(holdout_RF_path+'RF_holdout_results.csv')
# holdout_RF = holdout_RF.drop(columns=['prediction_type'])
original_RF = pd.read_csv(og_RF+'RF_test_results_all_NEK.csv')
original_RF['set'] = 'original_RF'
holdout_RF['set'] ='literature_RF'
columns = ['model','NEK', 'set','strategy', 'feat_type', 'RF_type','model', 'cm', 'recall', 'precision', 'accuracy','specificity',
           'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1']

holdout_RF = holdout_RF[columns]
original_RF = original_RF[columns]

all_rf = pd.concat([original_RF, holdout_RF], axis=0)

In [225]:
# avg_rf_dfs = []
# for model in all_rf['set'].unique(): 
#     this_df = all_rf[all_rf['set'] == model] 
#     print(f"{this_df['set'].loc[0]}") 
#     f1s = average_metric(this_df, 'f1')
#     MCCs = average_metric(this_df, 'MCC')
#     BAs = average_metric(this_df, 'Balanced Accuracy')
#     ROC_AUCs = average_metric(this_df, 'ROC-AUC')
#     precs = average_metric(this_df, 'precision')
#     accs = average_metric(this_df, 'accuracy')
#     specs = average_metric(this_df, 'specificity')
#     recalls = average_metric(this_df, 'recall')
#     avg_df = pd.DataFrame({'set': model,'recall': recalls, 'accuracy': accs, 'precision': precs, 'specificity': specs, 'f1': f1s,
#                           'MCC': MCCs, 'Balanced Accuracy': BAs, 'ROC-AUC': ROC_AUCs})
#     avg_rf_dfs.append(avg_df)

In [226]:
# original_rf_avg = avg_rf_dfs[0]
# literature_rf_avg  = avg_rf_dfs[1]
# literature_rf_avg['NEK'] = holdout_RF['NEK'].unique()
# literature_rf_avg.insert(0, 'NEK', literature_rf_avg.pop('NEK'))
# original_rf_avg['NEK'] = original_RF['NEK'].unique()
# original_rf_avg.insert(0, 'NEK', original_rf_avg.pop('NEK'))
# avg_rf_df_all = pd.concat([original_rf_avg, literature_rf_avg])
# avg_rf_df_all = avg_rf_df_all.sort_values('NEK')
# avg_rf_df_all['modeling_type'] = 'RF'
# print('RF original vs. literature test set (average of all each NEK dataset)')
# avg_rf_df_all.to_excel('comparison_RF_literature_set_by_NEK.xlsx',index=False)
# avg_rf_df_all

In [227]:
# avg_rf_strategy_dfs = []
# col_order = ['NEK', 'set', 'strategy','recall', 'accuracy', 'precision', 'specificity', 'f1',
#        'MCC', 'Balanced Accuracy', 'ROC-AUC']
# for model in all_rf['set'].unique(): 
#     this_df = all_rf[all_rf['set'] == model] 
#     print(f"{this_df['set'].loc[0]}") 
#     avg_df = average_metric_by_type(this_df, 'strategy')
#     avg_df = avg_df[col_order]
#     # display(avg_df)
#     avg_rf_strategy_dfs.append(avg_df)
# og_rf_df = avg_rf_strategy_dfs[0]
# lit_rf_df = avg_rf_strategy_dfs[1]
# rf_grouped_strategy_avg =pd.concat([og_rf_df, lit_rf_df])
# rf_grouped_strategy_avg['modeling_type'] = 'RF'
# rf_grouped_strategy_avg = rf_grouped_strategy_avg.sort_values(['NEK', 'strategy'])
# rf_grouped_strategy_avg.to_excel('comparison_RF_literature_set_by_strategy.xlsx',index=False)
# rf_grouped_strategy_avg

In [228]:
# avg_rf_feat_dfs = []
# col_order = ['NEK', 'set', 'feat_type','recall', 'accuracy', 'precision', 'specificity', 'f1',
#        'MCC', 'Balanced Accuracy', 'ROC-AUC']
# for model in all_rf['set'].unique(): 
#     this_df = all_rf[all_rf['set'] == model] 
#     print(f"{this_df['set'].loc[0]}") 
#     avg_df = average_metric_by_type(this_df, 'feat_type')
#     avg_df = avg_df[col_order]
#     # display(avg_df)
#     avg_rf_feat_dfs.append(avg_df)
# og_rf_df_feat = avg_rf_feat_dfs[0]
# lit_rf_df_feat = avg_rf_feat_dfs[1]

# rf_grouped_feat_avg =pd.concat([og_rf_df_feat, lit_rf_df_feat])
# rf_grouped_feat_avg['modeling_type'] = 'RF'
# rf_grouped_feat_avg = rf_grouped_feat_avg.sort_values(['NEK', 'feat_type'])
# rf_grouped_feat_avg.to_excel('comparison_RF_literature_set_by_feat.xlsx',index=False)
# rf_grouped_feat_avg

In [229]:
# avg_rf_RFttype_dfs = []
# col_order = ['NEK', 'set', 'RF_type','recall', 'accuracy', 'precision', 'specificity', 'f1',
#        'MCC', 'Balanced Accuracy', 'ROC-AUC']
# for model in all_rf['set'].unique(): 
#     this_df = all_rf[all_rf['set'] == model] 
#     print(f"{this_df['set'].loc[0]}") 
#     avg_df = average_metric_by_type(this_df, 'RF_type')
#     avg_df = avg_df[col_order]
#     # display(avg_df)
#     avg_rf_RFttype_dfs.append(avg_df)
# og_rf_df_RF = avg_rf_RFttype_dfs[0]
# lit_rf_df_RF = avg_rf_RFttype_dfs[1]

# rf_grouped_RFtype_avg =pd.concat([og_rf_df_RF, lit_rf_df_RF])
# rf_grouped_RFtype_avg['modeling_type'] = 'RF'
# rf_grouped_RFtype_avg = rf_grouped_RFtype_avg.sort_values(['NEK', 'RF_type'])
# rf_grouped_RFtype_avg.to_excel('comparison_RF_literature_set_by_RFtype.xlsx',index=False)
# # rf_grouped_RFtype_avg

# RF Grid Search

In [262]:
holdout_RFGS_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/holdout_test/RF_GS/'
og_RFGS_path= '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_grid_search/rf_results/'
holdout_RFGS = pd.read_csv(holdout_RFGS_path+'RF_GS_holdout_results.csv')
holdout_RFGS = holdout_RFGS.drop(columns=['prediction_type'])
original_RFGS = pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/RF_gridsearch_test_results_all_NEK.csv')
original_RFGS['set'] = 'original_RFGS'
holdout_RFGS['set'] ='literature_RFGS'
columns = ['model','NEK', 'set','strategy', 'feat_type', 'RF_type','model', 'cm', 'recall', 'precision', 'accuracy','specificity',
           'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1']
holdout_RFGS = holdout_RFGS[columns]
original_RFGS = original_RFGS[columns]
all_rfgs = pd.concat([original_RFGS, holdout_RFGS], axis=0)

In [263]:
gp_comparison = pd.read_excel('comparison/comparison_GP_literature_set.xlsx')
gp_comparison['modeling_type'] = 'GP'

In [264]:
rf_all_avg = pd.concat([avg_rf_df_all,avg_rfgs_df_all]).reset_index(drop=True)

In [265]:
cols2 = ['NEK',
 'set', 'modeling_type',
 'recall',
 'accuracy',
 'precision',
 'specificity',
 'f1',
 'MCC',
 'Balanced Accuracy',
 'ROC-AUC']

In [266]:
rf_strat_avg = pd.concat([rf_grouped_strategy_avg,rfgs_grouped_strategy_avg]).reset_index(drop=True)
rf_feat_avg = pd.concat([rf_grouped_feat_avg,rfgs_grouped_feat_avg]).reset_index(drop=True)
rf_RFtype_avg = pd.concat([rf_grouped_RFtype_avg,rfgs_grouped_RFtype_avg]).reset_index(drop=True)
rf_strat_avg.to_excel('RF_RFGS_original_literature_strategy_avg.xlsx',index=False)
rf_feat_avg.to_excel('RF_RFGS_original_literature_feat_avg.xlsx',index=False)
rf_RFtype_avg.to_excel('RF_RFGS_original_literature_RF_type_avg.xlsx',index=False)

In [279]:
full_og_GP = pd.read_csv(og_GP+'GP_test_results_all_NEK.csv')
full_og_GP['set'] = 'original' 
full_og_GP['modeling_type']='GP_RBF'
col_order = ['model','NEK', 'set','modeling_type','feat_type', 'strategy','cm','recall', 'accuracy', 'precision', 'specificity', 'f1',
       'MCC', 'Balanced Accuracy', 'ROC-AUC']
matern_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_matern_kernel/'
matern_GP = pd.read_csv(f'{matern_path}GP_maternkern_test_results.csv')
matern_GP['set'] = 'original'
matern_GP['modeling_type'] = 'GP_matern'
matern_GP=matern_GP[col_order]
full_holdout_GP= pd.read_csv(holdout_GP+'GP_holdout_test_results.csv')
full_holdout_GP['set'] = 'literature' 
full_holdout_GP['modeling_type'] = 'GP_RBF'
full_og_GP=full_og_GP[col_order]
full_holdout_GP=full_holdout_GP[col_order]
full_GP = pd.concat([full_og_GP, full_holdout_GP, matern_GP])
full_GP = full_GP.replace('scaled', 'raw')
# full_GP.to_excel('GP_results_all_sets.xlsx',index=False)
# full_GP.to_csv('GP_results_all_sets.csv',index=False) #104 
gp_matern_rbf = full_GP[(full_GP['modeling_type']=='GP_RBF')|(full_GP['modeling_type']=='GP_matern')]
gp_matern_rbf = gp_matern_rbf[gp_matern_rbf['set']=='original']
gp_matern_rbf.shape
gp_matern_rbf

,model,NEK,set,modeling_type,feat_type,strategy,cm,recall,accuracy,precision,specificity,f1,MCC,Balanced Accuracy,ROC-AUC
0,NEK2_binding_moe_ADASYN_GP,NEK2_binding,original,GP_RBF,moe,ADASYN,"[268, 3, 8, 4]",0.333333,0.961131,0.571429,0.988930,0.421053,0.418102,0.661132,0.661132
1,NEK2_binding_mfp_ADASYN_GP,NEK2_binding,original,GP_RBF,mfp,ADASYN,"[271, 0, 10, 2]",0.166667,0.964664,1.000000,1.000000,0.285714,0.400918,0.583333,0.583333
2,NEK2_inhibition_moe_ADASYN_GP,NEK2_inhibition,original,GP_RBF,moe,ADASYN,"[374, 7, 13, 15]",0.535714,0.951100,0.681818,0.981627,0.600000,0.579098,0.758671,0.758671
3,NEK2_inhibition_mfp_ADASYN_GP,NEK2_inhibition,original,GP_RBF,mfp,ADASYN,"[379, 2, 12, 16]",0.571429,0.965770,0.888889,0.994751,0.695652,0.697061,0.783090,0.783090
4,NEK3_binding_moe_ADASYN_GP,NEK3_binding,original,GP_RBF,moe,ADASYN,"[248, 17, 15, 2]",0.117647,0.886525,0.105263,0.935849,0.111111,0.050794,0.526748,0.526748
5,NEK3_binding_mfp_ADASYN_GP,NEK3_binding,original,GP_RBF,mfp,ADASYN,"[261, 4, 12, 5]",0.294118,0.943262,0.555556,0.984906,0.384615,0.377820,0.639512,0.639512
6,NEK5_binding_moe_ADASYN_GP,NEK5_binding,original,GP_RBF,moe,ADASYN,"[220, 8, 7, 13]",0.650000,0.939516,0.619048,0.964912,0.634146,0.601413,0.807456,0.807456
7,NEK5_binding_mfp_ADASYN_GP,NEK5_binding,original,GP_RBF,mfp,ADASYN,"[226, 2, 10, 10]",0.500000,0.951613,0.833333,0.991228,0.625000,0.623331,0.745614,0.745614
8,NEK9_binding_moe_ADASYN_GP,NEK9_binding,original,GP_RBF,moe,ADASYN,"[266, 4, 12, 1]",0.076923,0.943463,0.200000,0.985185,0.111111,0.098695,0.531054,0.531054
9,NEK9_binding_mfp_ADASYN_GP,NEK9_binding,original,GP_RBF,mfp,ADASYN,"[268, 2, 12, 1]",0.076923,0.950530,0.333333,0.992593,0.125000,0.142101,0.534758,0.534758


In [278]:
full_holdout_GP.shape

(56, 15)

In [277]:
4*2*6*2

96

In [283]:
full_holdout_RF = pd.read_csv(holdout_RF_path+'RF_holdout_results.csv')
full_holdout_RF['set'] = 'literature' 
full_holdout_RF['modeling_type'] = 'RF'
print("full holdout RF shape", full_holdout_RF.shape) # (224, 20)
print(full_holdout_RF['set'].unique())
RF_col_order = ['model','NEK', 'set','modeling_type','feat_type', 'strategy','RF_type','cm','recall', 'accuracy', 'precision', 'specificity', 'f1',
       'MCC', 'Balanced Accuracy', 'ROC-AUC']

original_RF = pd.read_csv(og_RF+'RF_test_results_all_NEK.csv')
print("original RF shape",original_RF.shape)
original_RF['set'] = 'original'
original_RF['modeling_type'] = 'RF'
full_holdout_RF=full_holdout_RF[RF_col_order]
original_RF=original_RF[RF_col_order] 

full_holdout_RFGS = pd.read_csv(holdout_RFGS_path+'RF_GS_holdout_results.csv')
full_holdout_RFGS['set'] = 'literature' 
full_holdout_RFGS['modeling_type'] = 'RF_GS'
original_RFGS = pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/RF_gridsearch_test_results_all_NEK.csv')
original_RFGS['set'] = 'original'
original_RFGS['modeling_type'] = 'RF_GS'
full_holdout_RFGS=full_holdout_RFGS[RF_col_order]
original_RFGS=original_RFGS[RF_col_order]
full_RF_RFGS = pd.concat([original_RF,full_holdout_RF,original_RFGS,full_holdout_RFGS])
# full_RF_RFGS.to_excel('RF_RFGS_results_all_sets.xlsx',index=False)
# full_RF_RFGS.to_csv('RF_RFGS_results_all_sets.csv',index=False)

all_all_GPRFRFGS = pd.concat([full_GP,full_RF_RFGS])
# all_all_GPRFRFGS = pd.concat([gp_matern_rbf,full_RF_RFGS]) # don't add holdout

all_all_GPRFRFGS.to_excel('GP_GPmatern_RF_RFGS_results_all_sets.xlsx',index=False)
all_all_GPRFRFGS.to_csv('GP_GPmatern_RF_RFGS_results_all_sets.csv',index=False)
all_all_GPRFRFGS.shape

full holdout RF shape (224, 20)
['literature']
original RF shape (192, 19)


(824, 16)

In [281]:
matern_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_matern_kernel/'
matern_GP = pd.read_csv(f'{matern_path}GP_maternkern_test_results.csv')
matern_GP['strategy']=matern_GP['strategy'].str.replace('scaled', 'raw')
matern_GP['model']=matern_GP['model'].str.replace('scaled', 'raw')
matern_GP['set'] = 'original'
matern_GP['modeling_type'] = 'GP_matern'